<a href="https://colab.research.google.com/github/MiguelUrenaPliego/labTopoSAM/blob/main/segment_anything_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from google.colab import output
output.enable_custom_widget_manager()

In [2]:
%pip install segment-geospatial groundingdino-py leafmap localtileserver geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 14.8 MB/s eta 0:00:

In [3]:
import os
import geopandas
import leafmap
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

In [4]:
batch = False #Esta variable será True si más adelante procesamos la imagen por lotes

Creamos las carpetas vector y raster para guardar los resultados

In [5]:
!mkdir raster
!mkdir vector

Creamos el objeto mapa

Siempre que hagamos algo con el objeto m estaremos editando el mapa

In [6]:
m = leafmap.Map(center=[43.2294769, -7.5824514], zoom=17, height="800px") # center=[lat, lon]
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
m

Map(center=[43.2294769, -7.5824514], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

Guardamos en bbox los límites de tu selección

In [7]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = "No has seleccionado nada"
    print("No has seleccionado nada")
    #bbox = [-122.2659, 37.8682, -122.2521, 37.8741]
bbox

[-7.5857, 43.2304, -7.584, 43.2315]

Decargamos las imágenes en formato tif en máxima resolución (zoom = 20)

Valores de zoom de 13 a 17 dan resoluciones menores pero aceptables y el modelo tardará menos en procesar, especialmente para la máscara automática (3).

In [8]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 01/30
Downloaded image 02/30
Downloaded image 03/30
Downloaded image 04/30
Downloaded image 05/30
Downloaded image 06/30
Downloaded image 07/30
Downloaded image 08/30
Downloaded image 09/30
Downloaded image 10/30
Downloaded image 11/30
Downloaded image 12/30
Downloaded image 13/30
Downloaded image 14/30
Downloaded image 15/30
Downloaded image 16/30
Downloaded image 17/30
Downloaded image 18/30
Downloaded image 19/30
Downloaded image 20/30
Downloaded image 21/30
Downloaded image 22/30
Downloaded image 23/30
Downloaded image 24/30
Downloaded image 25/30
Downloaded image 26/30
Downloaded image 27/30
Downloaded image 28/30
Downloaded image 29/30
Downloaded image 30/30
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


### Opción 2: Cargar imágenes desde archivo local
Puedes cargar tu propia imagen escribiendo la ruta. Debe ser un formato .tif con 3 canales R (rojo) G (verde) B (azul)

In [ ]:
# image = '/path/to/your/own/image.tif'

###Vemos las imágenes cargadas sobre el mapa

In [12]:
#Borra todo
m.clear_layers()
#Añade mapas básicos
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
#Si da error vuelve a ejecutar la celda de nuevo
#m.layers[1].visible = True #Para quitar la imagen satelite online
m.layers[-1].visible = False
m.remove_layer('Image')
m.add_raster(image, layer_name="Image")
m

Map(bottom=49200654.0, center=[43.23094999999999, -7.584849999999999], controls=(ZoomControl(options=['positio…

###Opcinal: *separar imágenes grandes en lotes*

In [ ]:
split_raster(image, out_dir="tiles", tile_size=(1000, 1000), overlap=0)
batch = True

#1 Segmentar a partir de texto

## Inicialización de SAM
SAM segment anything es un modelo de segmentación de imágenes desarrollado por meta https://github.com/facebookresearch/segment-anything . Tiene una versión especial para imágenes satélite https://samgeo.gishub.org/ .

Para que el procesamiento sea rápido requiere una GPU. En google collab lo hemos configurado ya en el primer paso.

Descargamos SamGeo (Puede tardar unos minutos dependiendo de tu internet)

Inicialización del modelo

In [ ]:
sam = LangSAM()

Especifica la clase que quieres segmentar

Puedes escribir cualquier cosa en inglés

In [ ]:
text_prompt = "building"

## Segmentación de la imagen

El resultado se guarda en el formato GEOtiff

box_threshold [0,1] tolerancia de deteccion. Valores altos dan lugar a detecciones muy seguras pero probablemente se pasen por alto bastantes elementos. valores bajos dan lugar a detecciones menos seguras con riesgo de detectar mal algunos elementos.

text_threshold [0,1] El modelo interpreta el texto que le damos para buscar la clase necesaria. Se puede escribir en text_promt cualquier cosa. Valores altos de text_threshold dan lugar a mayor exigencia del modelo para encontrar clases parecidas al texto dado. Valores bajos daran lugar a coincidencias más generales.

In [ ]:
box_threshold = 0.24
text_threshold = 0.24

In [ ]:
if batch: # Si procesamos por lotes
  sam.predict_batch(
    images='tiles',
    out_dir='tiles/text_mask',
    text_prompt=text_prompt,
    box_threshold=box_threshold,
    text_threshold=text_threshold,
    mask_multiplier=255,
    dtype='uint8',
    merge=True,
    verbose=True
    )
  !cp tiles/text_mask/merged.tif raster/text_mask.tif
else: #Si procesamos una única imagen
  sam.predict(image, text_prompt, box_threshold=box_threshold, text_threshold=text_threshold)
  sam.show_anns(
      cmap='Greys_r',
      add_boxes=False,
      alpha=1,
      blend=False,
      output='raster/text_mask.tif',
  )

#2 Segmentar a partir de puntos
Seleccionar al menos un punto dentro de cada objeto que quieres segmentar  

(a) Metadatos de SAM

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86, #precisión para considerar detección en tanto por 1
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

(b) Usar valores por defecto

In [ ]:
sam_kwargs = None

Inicialización de SAM

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint='sam_vit_h_4b8939.pth',
    sam_kwargs=sam_kwargs,
    automatic=False,
    device = 'cuda' #para trabajar en GPU Nvidia con cuda
)

In [ ]:
sam.set_image(image)

(a) Se le puede pasar una lista de puntos

In [ ]:
point_coords = [[-7.582178, 43.230454], [-7.583368, 43.229336], [-7.583701, 43.230102]]

(b) O puedes poner puntos en el mapa

In [ ]:
m

Guarda los puntos del mapa en el array point_coords

In [ ]:
m.save_draw_features("points.geojson")
coords = geopandas.read_file("points.geojson")['geometry'].get_coordinates(index_parts=True)
j = 0
point_coords = []
for i in geopandas.read_file("points.geojson")['geometry'].geom_type:
  if i is 'Point':
    point_coords.append([coords['x'][j][0],coords['y'][j][0]])
  j += 1
point_coords

###Segmentación
Le pedimos al modelo que segmente a partir de los puntos dados

In [ ]:
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="raster/point_mask.tif", batch = batch)

###Alternativa
Como alternativa se pueden seleccionar los puntos de manera interactiva en este mapa interactivo (No funciona muy bien)

In [ ]:
map_sel_points = sam.show_map()
map_sel_points

#3 Máscara automática
Segmenta todo lo que hay en la imagen

(a) Metadatos de SAM

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86, #precisión para considerar detección en tanto por 1
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

(b) Para usar los valores por defecto


In [ ]:
sam_kwargs = None

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint='sam_vit_h_4b8939.pth',
    sam_kwargs=sam_kwargs,
    automatic=True,
    device = 'cuda' #para trabajar en GPU Nvidia con cuda
)

###Generación automática de la máscara

El resultado se guarda en el formato GEOtiff

unique = True hace que a cada objeto segmentado se le asigne un identificador distinto

Suele tardar bastante. Se recomienda usar una imagen con menor resolución. (zoom = 17)

In [ ]:
sam.generate(image, output="raster/auto_mask.tif", foreground=True, unique=True, batch = batch)

Ver elementos segmentados con colores distintos

In [ ]:
if batch:
  print("Si se procesa por lotes esto no funciona")
sam.show_anns(axis="off", alpha=1, output="raster/auto_mask_annotations.tif")

#Visualización de resultados

Añadimos la máscara a nuestro mapa

*Si la leyenda y sus controles no funcionan bien cerrar la leyenda y volver a abrirla*

In [ ]:
#Borra todo
m.clear_layers()
#Añade mapas básicos
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
m.add_raster(image, layer_name="Image")
#Añade las máscaras raster generadas por sam. Lee la carpeta raster y añade todos los archivos que haya dentro.
for f in os.scandir('raster/'):
  if ".tif" in str(f):
    m.add_raster(f.path, cmap='viridis', nodata=0, layer_name="{}_raster".format(f.name.replace(".tif","")))
#m.add_layer_manager()
m.update_layer_manager()
m #Si no funciona bien volver a hacer click en la celda

###Formato Vectorial
Convertimos la máscara a vector

In [ ]:
for f in os.scandir('raster/'):
  if ".tif" in str(f):
    sam.raster_to_vector(f.path, "vector/{}".format(f.name.replace(".tif",".shp")))

Añadimos segmentación en formato vectorial al mapa

In [ ]:
#Borra todo
m.clear_layers()
#Añade mapas básicos
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
m.add_raster(image, layer_name="Image")
#Añade las máscaras vectoriales generadas por sam. Lee la carpeta vecgtor y añade todos los archivos que haya dentro.
for f in os.scandir('raster/'):
  if ".tif" in str(f):
    m.add_vector(f.path, cmap='Reds', nodata=0, layer_name="{}_vector".format(f.name.replace(".tif","")))
m.update_layer_manager()
m

Vector y raster junto

In [ ]:
#Borra todo
m.clear_layers()
#Añade mapas básicos
m.add_basemap("SATELLITE")
m.add_basemap("HYBRID")
m.add_raster(image, layer_name="Image")
#Añade las máscaras raster generadas por sam. Lee la carpeta raster y añade todos los archivos que haya dentro.
for f in os.scandir('raster/'):
  if ".tif" in str(f):
    m.add_raster(f.path, cmap='viridis', nodata=0, layer_name="{}_raster".format(f.name.replace(".tif","")))
#Añade las máscaras vectoriales generadas por sam. Lee la carpeta vecgtor y añade todos los archivos que haya dentro.
for f in os.scandir('raster/'):
  if ".tif" in str(f):
    m.add_vector(f.path, cmap='Reds', nodata=0, layer_name="{}_vector".format(f.name.replace(".tif","")))
m.update_layer_manager()
m

###Comparar imágenes


In [ ]:
archivo = "raster/text_mask.tif" #cambiar nombre para ver otra de las máscaras
leafmap.image_comparison(
    "satellite.tif",
    archivo,
    label1="Satellite Image",
    label2="Image Segmentation",
)